In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# we have an agent 
# the agent will be able to move through a matrix
#the matrix is a 10x15 space matrix
# there are locks thet the agent cannot move on to, these blocks are marked with an x
#agents can move up down left and right
# we need a history table called history with 3 columns, x position, y position and the reweard/punishment

agent= {"x":1,"y":-1}
table=[]
table.append(agent.copy())
action= ["Moved down"]
learning_rate = 0.1
gamma= 0.95
e_greedy=0.1


rewards=np.ones([10,15])
rewards=rewards*-0.01
rewards[9][12]= 1

actions=["Moved up","Moved down","Moved left","Moved right"]

q_table=[[0,0,"none",0]]

#actions for each state
for actionz in actions:
    # y values
    for i in range(1,11):
        # x values
        for j in range(1,16):
            #append all three witha  value of 0 for initialization
            q_table.append(list([j,i,actionz,0]))
q_table.pop(0)
q_table= pd.DataFrame.from_records(q_table)

In [ ]:

#functions to move
# current position of an agent is (x_t,y_t)
def move_up(agent):
    agent["y"]+=1   
    print("Moved up")
    action.append("Moved up")
    return(agent)
def move_down(agent):
    agent["y"]-=1  
    print("Moved down")
    action.append("Moved down")
    return(agent)
def move_right(agent):
    agent["x"]+=1   
    print("Moved right")
    action.append("Moved right")
    return(agent)
def move_left(agent):
    agent["x"]-=1   
    print("Moved left")
    action.append("Moved left")
    return(agent)

# function for hitting a wall
def hit_wall(agent,table):
    agent= table[-1]
    print("hit wall")
    return(agent)

# create a table of rewards for each point in the maze
end_point= [13,-10]
wall_points=[[3,-2],[3,-3],[3,-4],[1,-6],[2,-6],[3,-6],[4,-6],[5,-6],[6,-6],[7,-6],[8,-6]
             ,[14,-2],[14,-3],[14,-4],[14,-5],[14,-6],[14,-7],[10,-9],[11,-9],[12,-9],[13,-9],[14,-9],[15,-9],[7,-7],[7,-8]]

# create a function to implemant the action choices
def move_bi(agent,q_table, rand,rand2,end_point=end_point,wall_points=wall_points,table=table, greedy = 0.1,gamma=0.95):

    if rand2 >0 and rand2<=greedy:
        if rand >=0 and rand<=1 :
            agent=move_up(agent)
        if rand >=0.25 and rand<0.5 :
            agent=move_down(agent)
        if rand >=0.5 and rand<0.75 :
            agent=move_left(agent)
        if rand >=0.75 and rand<1 :
            agent= move_right(agent)
    else: 
    #greedy algo
        indices = [i for i, x in enumerate(q_table[(q_table[0]==agent["x"]) & (q_table[1]==-agent["y"])][3]) if x == max(q_table[(q_table[0]==agent["x"]) & (q_table[1]==-agent["y"])][3])]
        chosen_action=list(q_table[(q_table[0]==agent["x"]) & (q_table[1]==-agent["y"])][2])[random.choice(indices)]

        if chosen_action == "Moved up":
            agent=move_up(agent)
        elif chosen_action == "Moved down":
            agent=move_down(agent)
        elif chosen_action == "Moved left":
            agent=move_left(agent)
        elif chosen_action == "Moved right":
            agent=move_right(agent)

    return(agent)


def move_bi_action(agent,q_table, rand,rand2,end_point=end_point,wall_points=wall_points,table=table, greedy = 0.1,gamma=0.95):
    #print(rand)
    #print(rand2)
    if rand2 >0 and rand2<=greedy:
        #rand=random.random()
        if rand >=0 and rand<=1 :
            chosen_action = "Moved up"
        if rand >=0.25 and rand<0.5 :
            chosen_action = "Moved down"
        if rand >=0.5 and rand<0.75 :
            chosen_action = "Moved left"
        if rand >=0.75 and rand<1 :
            chosen_action = "Moved right"
    else: 
    #greedy algo
        indices = [i for i, x in enumerate(q_table[(q_table[0]==agent["x"]) & (q_table[1]==-agent["y"])][3]) if x == max(q_table[(q_table[0]==agent["x"]) & (q_table[1]==-agent["y"])][3])]
        chosen_action=list(q_table[(q_table[0]==agent["x"]) & (q_table[1]==-agent["y"])][2])[random.choice(indices)]

    return(chosen_action)

## Training epochs

In [ ]:
%matplotlib inline
epoch=0
n_iter=1000
steps=[0]
solved=0
while epoch<n_iter:
    solved=0
    step=0
    agent= {"x":1,"y":-1}
    table=[]
    table.append(agent.copy())
    action= ["Moved down"]
    rand1=random.random()
    rand2=random.random()
    print(agent)
    chosen_action=move_bi_action(agent,q_table, rand1, rand2,end_point,wall_points,table,greedy=e_greedy) 
    table.append(agent.copy())
    #call the function to return the first value for move here - init move value
    while solved==0:
        rand1=random.random()
        rand2=random.random()
        #print(q_table[(q_table[1]==-agent["y"] ) & (q_table[0]==agent["x"] )])
        if chosen_action == "Moved up":
            agent=move_up(agent)
        elif chosen_action == "Moved down":
            agent=move_down(agent)
        elif chosen_action == "Moved left":
            agent=move_left(agent)
        elif chosen_action == "Moved right":
            agent=move_right(agent)
        
        
        if [agent["x"],agent["y"]] in wall_points:
            agent=hit_wall(agent,table)

        if agent["x"]>15:
            agent=hit_wall(agent,table)

        if agent["x"]<=0:
            agent=hit_wall(agent,table)

        if agent["y"]<-10:
            agent=hit_wall(agent,table)
    
        if agent["y"]>=0:
            agent=hit_wall(agent,table)
        #make the current value for move

        if [agent["x"],agent["y"]] == end_point:
            #agent["reward"]=1
            print("solved")
            solved=1
            #break
        
        #update the next move
        chosen_action=move_bi_action(agent,q_table, rand1, rand2,end_point,wall_points,table,greedy=e_greedy) 

        table.append(agent.copy())
        
        Q_s_a=q_table[(q_table[1]==-table[-2]["y"]) & (q_table[0]==table[-2]["x"]) &(q_table[2]==action[-1])][3]
        #Q_s_a=[element[3] for element in q_table if element[2]==action[-1] and element[0]==table[-1]["x"] 
        #and element[1]== -table[-1]["y"]]
        #this needs to happen after the agent has moved and while updating the Q_value
        max_q_sa_next=q_table[(q_table[1]==-agent["y"] ) & (q_table[0]==agent["x"] )&(q_table[2]==chosen_action)][3]

        # Updating Q-values
        q_table[3][(q_table[1]==-table[-2]["y"]) & (q_table[0]==table[-2]["x"]) &(q_table[2]==action[-1])]=\
        Q_s_a + learning_rate*(rewards[-table[-2]["y"]-1][table[-2]["x"]-1] +(gamma* max_q_sa_next.values[0]) - Q_s_a)
        
        step+=1
#         print(epoch)
#         print(agent)

    epoch+=1
    
    steps.append(step)




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create data
x = [i for i in range(1,100)]
y = steps[1:100]
colors = (0,0,0)
#area = np.pi*3

# Plot
plt.plot(x, y, c=colors, alpha=1)
plt.title('Number of steps per Episode')
plt.xlabel('Episode')
plt.ylabel('Number of steps')
plt.show()

In [ ]:
import numpy as np
import matplotlib.colors as mlc
import matplotlib.pyplot as mlp

states = table_df["y"]
walk = table_df["x"]

states_len = len(states)
walk_len = len(walk)

img = np.zeros((states_len, walk_len), dtype=float)

for i, s in enumerate(walk):
    img[s, i] = 1.0

figure, ax = mlp.subplots()

color_map = mlc.LinearSegmentedColormap.from_list('ColorMap', [(1.000, 1.000, 1.000), (0.984, 0.501, 0.447)])
ax.imshow(img, cmap=color_map, interpolation='none')

ax.set_xlabel('Steps', fontsize=13)
ax.set_xticks(np.arange(0, walk_len, 1))
ax.set_xticks(np.arange(-0.5, walk_len, 1), minor=True)
ax.set_xticklabels(np.arange(1, walk_len + 1, 1))

ax.set_ylabel('States', fontsize=13)
ax.set_yticks(np.arange(0, states_len, 1))
ax.set_yticks(np.arange(-.5, states_len, 1), minor=True)
ax.set_yticklabels(states)

ax.grid(which='minor', color='k')

ax.set_title('Walkplot (Sequence)', fontsize=15, fontweight='bold')

## Final run after training

In [ ]:
# Just run for a single iteration after training
%matplotlib inline
epoch=0
n_iter=1
#steps=[0]
solved=0
while epoch<n_iter:
    solved=0
    #step=0
    agent= {"x":1,"y":-1}
    table=[]
    table.append(agent.copy())
    action= ["Moved down"]
    while solved==0:
        rand1=random.random()
        rand2=random.random()
        agent=move_bi(agent,q_table, rand1, rand2,end_point,wall_points,table,greedy=0)

        if [agent["x"],agent["y"]] in wall_points:
            agent=hit_wall(agent,table)

        if agent["x"]>15:
            agent=hit_wall(agent,table)

        if agent["x"]<=0:
            agent=hit_wall(agent,table)

        if agent["y"]<-10:
            agent=hit_wall(agent,table)

        if agent["y"]>=0:
            agent=hit_wall(agent,table)

        if [agent["x"],agent["y"]] == end_point:
            #agent["reward"]=1
            print("solved")
            solved=1
            #break

        #else: agent["reward"]=-0.01

        table.append(agent.copy())
        Q_s_a=q_table[(q_table[1]==-table[-2]["y"]) & (q_table[0]==table[-2]["x"]) &(q_table[2]==action[-1])][3]

        #this needs to happen after the agent has moved and while updating the Q_value
        max_q_sa_next=q_table[(q_table[1]==-agent["y"] ) & (q_table[0]==agent["x"] )][3].max()

        #print(q_table[(q_table[0]==1)&(q_table[1]==1)])
        # Updating Q-values
        q_table[3][(q_table[1]==-table[-2]["y"]) & (q_table[0]==table[-2]["x"]) &(q_table[2]==action[-1])]=\
        Q_s_a + learning_rate*(rewards[-table[-1]["y"]-1][table[-1]["x"]-1] +(gamma* max_q_sa_next) - Q_s_a)
        
        step+=1
#         print(epoch)
#         print(agent)

    epoch+=1
    
    #steps.append(step)




In [ ]:
import matplotlib
matplotlib.pyplot.grid(b=True, which='major', axis='both')
matplotlib.pyplot.grid(b=True, which='minor', axis='both')
plt.scatter(pd.DataFrame(table)["x"],pd.DataFrame(table)["y"])
plt.plot(pd.DataFrame(table)["x"],pd.DataFrame(table)["y"])
plt.scatter(pd.DataFrame(wall_points)[0],pd.DataFrame(wall_points)[1],color='red')
plt.yticks([i for i in range(-11,1)])
plt.xticks([i for i in range(0,16)])

plt.title('Agent Path')
plt.xlabel('X-axis')
plt.ylabel('y-axis')